# Natural Language Processing 2023-1A Homework 3

# Vector Semantics and Word2Vec 

Deadline: 27 September (23:59)

Questions: Post them in the HW3 discussion on Canvas, sent them to nlp-course@utwente.nl or ask us during the practical sessions. 

How to submit: Please answer the questions directly in this notebook and submit it before the deadline.

## Please Write your group number, your names with student IDs Here: 
Group 10 

Student Name and student Id: Hafsa Azhar s3099849




Make sure that the following libraries are up-to-date in your computation envrionment. It is highly recommended to work on this assignment in UT's [JupyterLab](https://www.utwente.nl/en/service-portal/research-support/it-facilities-for-research/jupyterlab). 

In [1]:
!pip3 install gensim nltk sklearn numpy pandas scipy 
!pip install  --upgrade gensim nltk sklearn numpy pandas scipy ### Upgrade your libraries if neccesary

ERROR: Invalid requirement: '###'


We'll need these libraries later.

In [2]:
import pandas, numpy, scipy, math
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity 
from gensim.test.utils import datapath
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import codecs

In this assignment, you will explore two types of word vectors: those derived from **co-occurrence count-based methods**, and those derived via the **local context predictive model word2vec**. 

Note on Terminology: 
- The word "word" and "term" are used interchangeably here. They both mean the unique tokens that you would like to represent in terms of vectors. Often they are individual words, n-grams,  phrases, or even identifiers. In the assignment, we focus on individual words. 
- The terms "word vectors" and "word embeddings" are often used interchangeably, but they are actually different. According to [Wikipedia](https://en.wikipedia.org/wiki/Word_embedding), conceptually, word embedding "*involves the mathematical embedding from space with many dimensions per word to a continuous vector space with a much lower dimension*".

# Part I. Co-occurrence count-based vectors

Let's start with this corpus consisting of 5 sentences.

In [3]:
sents=['Simultaneous heatwaves at North and South Poles.',
        'Both North and South poles see unusual high heat.',
        'North pole is heating up. What about South pole?',
        'Bizarre heatwaves strike Arctic and Antarctic poles.',
        'Climate change at the poles.'
       ]

### Exercise 1.1.1 Construct the vocabulary (0.5 point)
Before we construct co-occurrence matrices, we need to identify unique terms in the corpus, i.e. construct the vocabulary. You can remove stop words and apply other text normalisation operations before constructing the vocabulary. 

Tip: Sort your vocabulary alphabetically!

In [4]:
# Creating a CountVectorizer with stop words and sorting it
vectorizer = CountVectorizer(stop_words=stopwords.words('english')).fit(sents)
vocab = [*vectorizer.vocabulary_]
vocab = sorted(vocab, reverse=False)

# your code ends here

print('The size of the vocabulary is', len(vocab))
print('The words in the vocabulary are', vocab)


The size of the vocabulary is 17
The words in the vocabulary are ['antarctic', 'arctic', 'bizarre', 'change', 'climate', 'heat', 'heating', 'heatwaves', 'high', 'north', 'pole', 'poles', 'see', 'simultaneous', 'south', 'strike', 'unusual']


### Co-Occurrence

A co-occurrence matrix counts how often terms co-occur in certain context. The context can be a complete document, a sentence, or a sliding window. 

Tip: Check out the [sklearn.feature_extraction.text](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text) submodule that gathers utilities to build feature vectors from text documents. 

### Exercise 1.1.2 Term-document occurrence matrix and term-term co-occurrence matrix (0.5 point)
Let's first consider **each sentence** in the above corpus to be the context where the (co-)occurrences are counted. For example, the words *heatwaves*, *north*, *south* and *poles* occur in the first sentence, therefore, they occur in this document and co-occur with each other. Going through all the sentences, you can construct the term-document occurrence matrix and term-term co-occurrence matrix. 

In [6]:
# Construct the term-document occurrence matrix: tdMatrix_pd

count_vectorizer= CountVectorizer(vocabulary=vocab, binary=True)
tdMatrix = count_vectorizer.transform(sents)
# to match the expected shape 
tdMatrix = tdMatrix.transpose()
# your code ends here

print('The shape of the term-document matrix is', tdMatrix.shape)
tdMatrix_pd = pandas.DataFrame(tdMatrix.toarray(), index=vocab, columns=range(1, len(sents) + 1))
tdMatrix_pd

The shape of the term-document matrix is (17, 5)


,1,2,3,4,5
antarctic,0,0,0,1,0
arctic,0,0,0,1,0
bizarre,0,0,0,1,0
change,0,0,0,0,1
climate,0,0,0,0,1
heat,0,1,0,0,0
heating,0,0,1,0,0
heatwaves,1,0,0,1,0
high,0,1,0,0,0
north,1,1,1,0,0


Tip: You can derive the term-term co-occurrence matrix directly from the term-document occurrence matrix. 

In [7]:
# your code starts here
# Convert tdMatrix to a NumPy array
tdMatrix = tdMatrix.toarray()

# Construct the term-term co-occurrence matrix
ttMatrix = tdMatrix.dot(tdMatrix.T)
numpy.fill_diagonal(ttMatrix, 0)
# your code ends here
print('The shape of the term-term matrix is', ttMatrix.shape)
ttMatrix_pd = pandas.DataFrame(ttMatrix, index=vocab, columns=vocab)
ttMatrix_pd


The shape of the term-term matrix is (17, 17)


,antarctic,arctic,bizarre,change,climate,heat,heating,heatwaves,high,north,pole,poles,see,simultaneous,south,strike,unusual
antarctic,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0
arctic,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0
bizarre,1,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
change,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
climate,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
heat,0,0,0,0,0,0,0,0,1,1,0,1,1,0,1,0,1
heating,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0
heatwaves,1,1,1,0,0,0,0,0,0,1,0,2,0,1,1,1,0
high,0,0,0,0,0,1,0,0,0,1,0,1,1,0,1,0,1
north,0,0,0,0,0,1,1,1,1,0,1,2,1,1,3,0,1


Based on term-term co-occurrence matrix, which pair(s) of words co-occur the most? 

**YOUR ANSWER**: 
"north" "south" are the the words that occurs the most

### Exercise 1.1.3 (1 point)
Given some word $w_i$ occurring in the corpus, we can also consider the **context window** surrounding $w_i$. Supposing our fixed window size is $n$, then this is the $n$ preceding and $n$ subsequent words in that document, i.e. words $w_{i-n} \dots w_{i-1}$ and $w_{i+1} \dots w_{i+n}$. We build a *co-occurrence matrix* $M$, which is a symmetric word-by-word matrix in which $M_{ij}$ is the number of times $w_j$ appears inside $w_i$'s window. 


In [8]:
from nltk.tokenize import RegexpTokenizer


def getTermTermWindowMatrix(corpus, window_size=2):
    """ Note: Apply text normalisation before counting. 
              Each word in a document should be at the center of a window. 
              Words near edges will have a smaller number of co-occurring words. 
              For example, for the document "Both North and South poles see unusually high heat" with window size of 4,
              "north" will co-occur with one word before (i.e. "both") and four words after (i.e. "south", "poles", "see", and "unusally")
    
        Params:
            corpus (list of list of strings): corpus of documents
            window_size (int): size of context window
        Return:
            M (numpy matrix of shape (size of vocabulary, size of vocabulary)): 
                Co-occurence matrix of word counts. 
                The vocabulary contains all the unique words in the corpus
                The ordering of the words in the rows/columns should be the same

    """
    # your code starts here
    vectorizer = CountVectorizer(stop_words=stopwords.words('english')).fit(corpus)
    vocab = [*vectorizer.vocabulary_]
    vocab = sorted(vocab, reverse=False)
    #Initialize it with zeros
    tmMatrix = numpy.zeros((len(vocab), len(vocab)))
    #check each document 
    for doc in corpus:
        tokens = RegexpTokenizer(r'\w+').tokenize(doc.lower())
        tokens = [word for word in tokens if not word in stopwords.words('english')]  #if not then put in the tokens
        #Now check each document and each words according to critea within the window size 
        for i in range(len(tokens)): #check each words
            for j in range(window_size): #check according to the window size
                if i - j - 1 >= 0 and tokens[i] not in stopwords.words('english') and tokens[i - j - 1] not in stopwords.words('english'):
                    tmMatrix[vocab.index(tokens[i])][vocab.index(tokens[i - j - 1])] += 1 #check the center word and before if it is in limit then increment
                if i + j + 1 < len(tokens) and tokens[i] not in stopwords.words('english') and tokens[i + j + 1] not in stopwords.words('english'):
                    tmMatrix[vocab.index(tokens[i])][vocab.index(tokens[i + j + 1])] += 1 #Now check the center word and after if it is in limit then increment

    return tmMatrix, vocab
    # your code ends here
ttwMatrix,vocab=getTermTermWindowMatrix(sents, 2)
seettwMatrix = pandas.DataFrame(ttwMatrix, index=vocab, columns=vocab)
seettwMatrix

,antarctic,arctic,bizarre,change,climate,heat,heating,heatwaves,high,north,pole,poles,see,simultaneous,south,strike,unusual
antarctic,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
arctic,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
bizarre,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
change,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
climate,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
heat,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
heating,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
heatwaves,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
high,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
north,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,2.0,0.0,0.0


## Exercise 1.2 Cosine similarity
The benefit of vector semantics is that the similarity of two words can be computed as the cosine similarity between their vectors. Let's now compare how similar two words are. 

### Exercise 1.2.1 Write your own cosine similarity function (0.5 point)

The cosine similarity metric between two vectors $v$ and $w$  can be computed as:
$cosine(v,w )={\mathbf {v} \cdot \mathbf {w}  \over \|\mathbf {v} \|\|\mathbf {w} \|}={\frac {\sum \limits _{i=1}^{n}{v_{i}w_{i}}}{{\sqrt {\sum \limits _{i=1}^{n}{v_{i}^{2}}}}{\sqrt {\sum \limits _{i=1}^{n}{w_{i}^{2}}}}}}$.

Implement your cosine similarity function. 

In [9]:
import numpy as np

# Define the cosine similarity function
def cosine_similarity(vec1, vec2):
    #Now lets check according to the forula given above
    return ((numpy.array(vec1)*numpy.array(vec2)).sum())/(math.sqrt((numpy.array(vec1)*numpy.array(vec1)).sum())*math.sqrt((numpy.array(vec2)*numpy.array(vec2)).sum()))


    

# Your code continues here
vec1 = tdMatrix[vocab.index('antarctic')]
vec2 = tdMatrix[vocab.index('arctic')]
print(cosine_similarity(vec1, vec2))

vec1 = ttMatrix[vocab.index('antarctic')]
vec2 = ttMatrix[vocab.index('arctic')]
print(cosine_similarity(vec1, vec2))

vec1 = ttwMatrix[vocab.index('antarctic')]
vec2=ttwMatrix[vocab.index('arctic')]
print(cosine_similarity(vec1, vec2))


1.0
0.7999999999999998
0.5773502691896258


What is the cosine similarity between "antarctic" and "arctic" if 1) using term-document occurrence matrix 2) using term-term co-occurrence matrix?

**Your Answer**
we got 1.0 for the term document, 0.799 for term-term co-occurrence matrix and 0.577 acording to the window size.



### Exercise 1.2.2 (0.5 point)

Now we can calculate cosine similarity between words using a co-occurrence matrix. You can choose any previously constructed matrix for the similarity calculation. Rank all the words based on their similarity to the word *north*.

In [10]:
# Rank all the words by their similarity to word "north"

# your code starts here
#First let's make a an empty set
similarity_cal = []
for word in vocab:
    if vocab.index("north") == vocab.index(word):
        similarity_cal.append(0.0) #cosine similarity between north and itself is always 1, and this code represents it as 0.0
    else:
            similarity_cal.append(cosine_similarity(tdMatrix[vocab.index('north')],tdMatrix[vocab.index(word)])) #calculates the similarity
ranking = pandas.DataFrame(similarity_cal, index=vocab, columns=["similarity_cal"])
ranking.sort_values('similarity_cal', ascending=False)
        
            
        



# your code ends here

,similarity_cal
south,1.000000
high,0.577350
pole,0.577350
simultaneous,0.577350
see,0.577350
poles,0.577350
unusual,0.577350
heating,0.577350
heat,0.577350
heatwaves,0.408248



Does  the  calculated  cosine  similarity  reflect  semantic  similarity  or  relatedness?
- If yes, to what extent?
- If not, think of ways to get more reliable similarities



**YOUR ANSWER**: 



Cosine similarity measures the cosine of the angle based on their co-occurrence patterns in a corpus. While it can provide some indication of relatedness, it doesn't capture the full depth of semantic similarity, especially in a small corpus. It also depends on the context window size, the smaller context window size might miss broader semantic relationships.



# Exercise 1.3 TF-IDF

## Excercise 1.3.1 (1 point)
For the above corpus, construct a TF-IDF weighted term-document matrix, using the formula 6.12 and 6.13 in the J&M book. 
- $tf(t,d) = \log _{10}(count(t,d) +1)$ (6.12)
- $idf(t) = \log_{10} \frac{N}{df(t)}$ (6.13)


In [11]:
#in order to calculate TF-IDF we need to calculate the frequency of words and 
#term frequency (tf) for the word in that document 


# Function to calculate TF-IDF matrix
def get_tfidfMatrix(corpus, vocab):
    M = len(vocab)
    N = len(corpus)
    tfidf_matrix = np.zeros((M, N), dtype=float)  # Initialize the matrix with zeros

    for i, word in enumerate(vocab):
        idft = math.log10(N / np.count_nonzero(tdMatrix[i, :]))  # Calculate IDF for the word

        for j in range(N):
            tf = math.log10(tdMatrix[i, j] + 1)  # Calculate TF for the word in the document

            # Calculate TF-IDF and store it in the matrix
            tfidf_matrix[i, j] = tf * idft

    return tfidf_matrix

tfidf_matrix = get_tfidfMatrix(sents, vocab)

tfidf_matrix_pd = pandas.DataFrame(tfidf_matrix, index=vocab, columns=[1,2,3,4,5])
# your code ends here


Check the ranked list for *north* and see whether it changes. 

In [12]:
# Rank all the words by their similarity to word "north"

# your code starts here
#First let's make a an empty set
similarity_cal = []
for word in vocab:
    if vocab.index("north") == vocab.index(word):
        similarity_cal.append(0.0) #cosine similarity between north and itself is always 1, and this code represents it as 0.0
    else:
            similarity_cal.append(cosine_similarity(tfidf_matrix[vocab.index('north')],tfidf_matrix[vocab.index(word)])) #calculates the similarity
ranking = pandas.DataFrame(similarity_cal, index=vocab, columns=["similarity_cal"])
ranking.sort_values('similarity_cal', ascending=False)
        
            
        



# your code ends here

,similarity_cal
south,1.000000
high,0.577350
pole,0.577350
simultaneous,0.577350
see,0.577350
poles,0.577350
unusual,0.577350
heating,0.577350
heat,0.577350
heatwaves,0.408248


**YOUR ANSWER**: 


## Exercise 1.3.2 (1 point)

Let's use a bigger dataset which contains 2225 BBC news articles to construct TF-IDF term-document matrix. You may use your own function or [`sklearn.feature_extraction.text.TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) from the scikit library to generate TF-IDF weighted term-document matrix. When your matrix is ready, you may use [`sklearn.metrics.pairwise.cosine_similarity`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) to calculate the pair-wise cosine similarity among all the words. 

In [13]:
#first lets load the data using pandas 
sents=pandas.read_csv('bbc-text.csv')
#Then make sents and vectorize it in to get vocabs and then sort it out
sents= sents['text']
vectorizer = CountVectorizer().fit(sents.tolist())
vocab = [*vectorizer.vocabulary_]
vocab = sorted(vocab, reverse=False)

tdoc_matrix = TfidfVectorizer().fit_transform(sents).toarray()
tdoc_matrix = numpy.matrix(tdoc_matrix).transpose()
tdoc_matrix = pandas.DataFrame(tdoc_matrix, index=vocab, columns=list(range(0, len(sents))))
tdoc_matrix = tdoc_matrix.to_numpy()

print('The size of the vocabulary is', len(vocab))
print('The shape of the term-document matrix is', tdoc_matrix.shape)


The size of the vocabulary is 29421
The shape of the term-document matrix is (29421, 2225)


We can compute which words are most similar to *north*. Does this list of words make more sense now and why?

In [14]:
# Find the top 10 words that are most similar to word "north"

# your code starts here
# Rank all the words by their similarity to word "north"

# your code starts here
#First let's make a an empty set
similarity_cal = []
for word in vocab:
    if vocab.index("north") == vocab.index(word):
        similarity_cal.append(0.0) #cosine similarity between north and itself is always 1, and this code represents it as 0.0
    else:
            similarity_cal.append(cosine_similarity(tdoc_matrix[vocab.index('north')],tdoc_matrix[vocab.index(word)])) #calculates the similarity
ranking = pandas.DataFrame(similarity_cal, index=vocab, columns=["similarity_cal"])
ranking = ranking.sort_values('similarity_cal', ascending=False)
ranking.head(10)
            
        



# your code ends here


,similarity_cal
mallon,0.441048
metric,0.441048
wades,0.441048
487,0.441048
ne,0.441048
robb,0.441048
exerts,0.441048
tomaney,0.441048
939,0.441048
herron,0.377049


**YOUR ANSWER**: 


Find another 3 pairs of words whose cosine similarity makes sense to you.

In [36]:
# Look for 3 pairs of words whose cosine similarities reflect their semantic similarity or relatedness.


home_index = vocab.index("tv")
house_index = vocab.index("television")
similarity1 = cosine_similarity(tdoc_matrix[home_index].reshape(1, -1), tdoc_matrix[house_index].reshape(1, -1))
print("The similarity between 'tv' and 'television' is", similarity1)

technology_index = vocab.index("technology")
technologies_index = vocab.index("technologies")
similarity2 = cosine_similarity(tdoc_matrix[technology_index].reshape(1, -1), tdoc_matrix[technologies_index].reshape(1, -1))
print("The similarity between 'technology' and 'technologies' is", similarity2)

nintendo_index = vocab.index("nintendo")
ds_index = vocab.index("ds")
similarity3 = cosine_similarity(tdoc_matrix[nintendo_index].reshape(1, -1), tdoc_matrix[ds_index].reshape(1, -1))
print("The similarity between 'nintendo' and 'ds' is", similarity3)


The similarity between 'tv' and 'television' is [[0.33857192]]
The similarity between 'technology' and 'technologies' is [[0.41697637]]
The similarity between 'nintendo' and 'ds' is [[0.91591187]]


# Part II. Word2Vec word vectors

Here, we explore the embeddings produced by word2vec. Please read J&M 6.8 or the [original paper](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) if you are interested in the details of the algorithm.

## Exercise 2.1 Pre-train word2vec model

Run the following script to load the word2vec vectors into memory. **Note**: This might take several minutes. If you run out of memory, try closing other applicaions or restart your machine to free more memory. 

Please note, the following experiments run with Gensim 4.2.0. If you are still running an old version of Gensim, please upgrade your Gensim library or check [Migrating from Gensim 3.x to 4](https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4) to adapt your code. 

In [ ]:
# Load 3 million Word2Vec Vectors, pre-trained on Google news, each with the dimension of 300
# This model may take a few minutes to load.

import gensim.downloader as api
start_time = time.time()
w2v_google = api.load("word2vec-google-news-300")
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print("Loaded vocab size {}".format(len(w2v_google.index_to_key)))

Once the model is loaded, you can extract the vector for individual words directly using `w2v_google['']`

In [ ]:
w2v_google['north']

One of the property of semantic embedding is that similar words are embedded close to each other. Use  `w2v_google.most_similar()` to identify the most similar words to *north*. Does this list make more sense to you? 

In [ ]:
start_time = time.time()
for w,c in w2v_google.most_similar('north'):
    print(w,c)
print("--- %s seconds ---" % (time.time() - start_time))

Check a few more words to see whether their most similar words make sense to you and explain why. 

In [ ]:
# Check whether the computed most similar words make sense 
w2v_google.most_similar('WORD1')
w2v_google.most_similar('WORD2')
w2v_google.most_similar('WORD3')

**YOUR ANSWER**: 


## Word analogies

An analogy explains one thing in terms of another to highlight the ways in which they are alike. For example, *paris* is similar to *france* in the same way that *rome* is to *italy*. Word2Vec vectors sometimes shows the ability of solving analogy problem of the form **a is to b as a* is to what?**.

In the cell below, we show you how to use word vectors to find x. The `most_similar` function finds words that are most similar to the words in the `positive` list and most dissimilar from the words in the `negative` list. The answer to the analogy will be the word ranked most similar (largest numerical value). In the case below, the top one word *italy* is the answer, so this analogy is solved successfully.

In [ ]:
# Run this cell to answer the analogy -- paris : france :: rome : x
print(w2v_google.most_similar(positive=['rome', 'france'], negative=['paris']))

### Exercise 2.1.1 (0.5 point)
Look for one analogy that can be solved successfully and one analogy that could not be solved using this pre-trained Word2Vec model. Check out [this paper](https://www.semanticscholar.org/paper/Efficient-Estimation-of-Word-Representations-in-Mikolov-Chen/330da625c15427c6e42ccfa3b747fb29e5835bf0) for inspirations.

In [ ]:
# successful case


# failed case



## Visualising word analogies

The following cell shows you how to use [tSNE](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html) to visualise a set of words based on their embeddings. You can also apply other dimensionality reduction methods (e.g. [sklearn.decomposition.TruncatedSVD](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html)) to reduce the vectors from 300-dimensional to 2 dimensional. 

Please note, reducing dimensionality from 300 to 2 is a very challenging task. You can try different parameters in the tSNE and see their effects on the final visualisation. In particular, the visualisation is very sensitive to the perplexity value that you give. Please try a few different perplexity valuse and keep the one that gives the most reasonable visusalisation. 

In [ ]:
def tsne_plot(model, wordlist,p): # Create TSNE model and plot it
    labels = []
    tokens = []
    
    for word in wordlist:
        tokens.append(model[word])
        labels.append(word)
    
    tokens=numpy.array(tokens)
    
    tsne_model = TSNE(perplexity=p, n_components=2, init='pca', n_iter=2500, random_state=23) 
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(18, 18)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()
   
wordlist=['beijing','china', 'london', 'uk', 'berlin', 'germany', 'rome', 'italy','tokyo', 'japan', ]
tsne_plot(w2v_google, wordlist,len(wordlist)-1)

### Exercise 2.1.2 (0.5 point)
Find another group analogies (at least 3 pairs of words) and see how they are visualised.  

In [ ]:
# prepare at least 3 pairs of words   

# your answer goes here

wordlist=[]
p=len(wordlist)-1
tsne_plot(w2v_google,wordlist,p)

### Exercise 2.1.3  Synonyms and antonyms (0.5 point)



Find three words (w1, w2, w3) so that 
- w1 and w2 are synonyms, 
- w1 and w3 are antonyms, 
- cosine_distance(w1, w2) > cosine_distance(w1, w3) or cosine_distance(w1, w2) $\approx$ cosine_distance(w1, w3). 

Please give a possible explanation for why this has happened. 

You can use [`w2v_google.distance()`](https://radimrehurek.com/gensim/models/keyedvectors.html) function to compute the cosine distance between two words. 

In [ ]:
w1=''
w2=''
w3=''

print("Synonyms {}, {} have cosine distance: {}".format(w1, w2, w2v_google.distance(w1, w2)))
print("Antonyms {}, {} have cosine distance: {}".format(w1, w3, w2v_google.distance(w1, w3)))


**Your answer**: 

Word2Vec and other vector space models use co-occurrences as the evidence for computing the semantic similarity and relatedness. Some antonyms are often used together in the same context, therefore their vector representations or embeddings are numerically close to each other (small distance). However, many synonyms  are not used often in the same textual context therefore will not have similar vectors, i.e. the cosine distance is higher.  

### Exercise 2.1.4 Polysemous Words (0.5 point)

Some words are polysemous, i.e. they have multiple meanings. For example the word *bank* can be a financial institute or the rising ground bordering a lake or river. Find a polysemous word whose top most similar words contains related words from multiple meanings. You should use the the [`wv_google.most_similar()`](https://radimrehurek.com/gensim/models/keyedvectors.html) function to compute the closet neighbours of the word. You may increase the number of neighbours in order to identify multiple groups of meanings. Submit the ranked word list and explained how the words are grouped into different meanings.

In [ ]:
w2v_google.most_similar('run',topn=20)

In [ ]:
focusword='run'
wordlist=[focusword]
for w in w2v_google.most_similar(focusword,topn=100):
    wordlist.append(w[0])
print(wordlist)
tsne_plot(w2v_google,wordlist,50)

Look into literature and describe potential methods to address this polysymy issue in word embeddings. Please cite the papers that you refer to. 

**YOUR ANSWER**:


## Exercise 2.2  Self-trained Word2Vec model

The word2vec model that we have been using so far is pre-trained on Google news. This is suitable for applications involving general topics. However, for special domains, such as scientific or medical domain, some domain-specific semantics could not be captured in the pre-trained model. Fortunately, word2vec is pretty efficient in training from scratch. We will use two different datasets to observer the effect on the input corpus. 

Importance parameters are highlighted in bold. Please choose a few different values and see their effects.  

class gensim.models.word2vec.Word2Vec(sentences=None, corpus_file=None, **vector_size=100**, alpha=0.025, **window=5**, **min_count=5**, max_vocab_size=None, sample=0.001, seed=1, workers=3, min_alpha=0.0001, **sg=0**, hs=0, **negative=5**, ns_exponent=0.75, cbow_mean=1, hashfxn=<built-in function hash>, epochs=5, null_word=0, trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False, callbacks=(), comment=None, max_final_vocab=None, shrink_windows=True)
    
Please check the [gensim documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec) for more assistance. 

In [ ]:
# the most similar words to 'young' in Google news
w2v_google.most_similar('young')

### Exercise 2.2.1 (1 point)

We first train a word2vec model on the corpus consisting the abstracts from 111K astrophysics/astronomy articles.

In [ ]:

# This might take up a few minutes to train.
from gensim.models.word2vec import LineSentence, Word2Vec
sentences=LineSentence('astro_norm.txt')

start_time = time.time()
# Train a word2vec model using the astro dataset
# your code starts here


# your code ends here
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
w2v_astro.wv.most_similar('young')

If all goes well, you may see *pms*, *proto* or *yso* among the top 10 most similar words to *young*. If you are curious, protostars and pre-main-sequence (PMS) stars are all [Young Stella Objects](https://en.wikipedia.org/wiki/Young_stellar_object)  (YSOs). Here, “young” means pre-main-sequence. For low-mass stars, this means ages of $10^5$ to $10^8$ years. [Ref](https://nexsci.caltech.edu/workshop/2003/2003_MSS/10_Thursday/mss2003_jensen.pdf)

We then train a word2vec model on the corpus consisting of nearly 479K [Medline](https://www.nlm.nih.gov/medline/medline_overview.html) articles. Note, this corpus is rather big. If this is too much for your local machine, use UT's [JupyterLab](https://www.utwente.nl/en/service-portal/research-support/it-facilities-for-research/jupyterlab) or [Google Colab](https://colab.research.google.com/notebooks/). 

In [ ]:
# This might take up half an hour to train!

from gensim.models.word2vec import LineSentence, Word2Vec
sentences=LineSentence('medline_norm.txt')

start_time = time.time()
# Train a word2vec model using the astro dataset
# your code starts here


# your code ends here
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
w2v_medline.wv.most_similar('young')

Find another word and compute its most similar words based on different models. If you get different lists of words that are most similar to the target word, please explain why this happens.

**YOUR ANSWER**:

In [ ]:
w2v_google.most_similar('')

In [ ]:
w2v_astro.wv.most_similar('')

In [ ]:
w2v_medline.wv.most_similar('')

**YOUR ANSWER:** 



### Exercise 2.2.2 (1 point)

Experiment with different parameters, for example, the vector size, the window size, the minimal count, skip-gram or CBOW, etc. Observe their effects on the quality of the word embeddings and/or computational cost. 

You can apply intrinsic evaluations to compare the quality of your models. For example, your can check the correlation with human opinion on word similarity or on word analogies. Check [gensim documentations](https://radimrehurek.com/gensim/models/keyedvectors.html) for more options.


**YOUR ANSWER**:

### Exercise 2.2.3 (0.5 point)
Name at least 2 advantages of dense word vectors such as those calculated with Word2Vec over the vectors based on the term-document or term-term matrix as in Part I.

**YOUR ANSWER**:


### Exercise 2.2.4 (0.5 point)
Name  at  least  2  limitations  of  the  Word2Vec  embedding  model.   Note,these limitations are not necessarily exclusive for Word2Vec.

**YOUR ANSWER**: